In [ ]:
import requests
import json
import os

In [ ]:
def get_fred_api_key_from_cfg(path:str='quantz_cfg.json'):
    try:
        with open(path, 'r') as f:
            config = json.load(f)
            return config['fred_api_key']
    except BaseException as e:
        print('Failed to get fred api key from %s cause %s' % (path, e))
        return None

get_fred_api_key_from_cfg()

In [ ]:
class Fred:
    '''
    直接返回获取的 json 或者 抛出异常提示错误
    '''
    def __init__(self):
        '''
        get Fred 数据
        TODO: 通过配置或环境变量获取 Api Key

        '''
        self.api_key = get_fred_api_key_from_cfg()
        if self.api_key is None:
            raise Exception('Fred Api Key can not be empty')
        self.path='https://api.stlouisfed.org/fred'

    def __get(self, args):
        print('__get:%s' % self.path)
        print(args)
        resp = None
        if len(args) == 0:
            resp = requests.get(self.path, params={'api_key':self.api_key, 'file_type':'json'})
        elif len(args) == 1:
             params={'api_key':self.api_key, 'file_type':'json'}
             params.update(args[0])
             print(params)
             resp = requests.get(self.path, params=params)
        else:
            print('Invalid args%s' % args)
        # TODO: 增加网络错误的处理
        # print(resp.json())
        return resp.json()

    def __getattr__(self, name):
        '''
        __getattr__ 在需要的属性没有找到的情况下会调用
        __getattribute__ 在读取任何属性的时候都会调用
        '''
        self.path = '%s/%s' % (self.path, name)
        return self

    def __str__(self):
        return self.path

    def __call__(self,*args):
        return self.__get(args)

In [ ]:
print(Fred().series({'series_id':'GNPCA', 'realtime_start':'2019-08-14', 'realtime_end':'2021-03-04'}))
print(Fred().sources())